In [1]:
import osmnx as ox

cities = [
    "Tijuana, Baja California, Mexico",
    "Mexicali, Baja California, Mexico",
    "Ensenada, Baja California, Mexico",
    "Tecate, Baja California, Mexico",
    "Rosarito, Baja California, Mexico",
    "San Quintin, Baja California, Mexico",
    "San Felipe, Baja California, Mexico",
]

coords = {city: ox.geocode(city) for city in cities}

coords

{'Tijuana, Baja California, Mexico': (32.5317397, -117.019529),
 'Mexicali, Baja California, Mexico': (32.6245314, -115.452604),
 'Ensenada, Baja California, Mexico': (31.8658887, -116.602983),
 'Tecate, Baja California, Mexico': (32.5653831, -116.6298787),
 'Rosarito, Baja California, Mexico': (32.365166, -117.0557923),
 'San Quintin, Baja California, Mexico': (30.5613719, -115.938162),
 'San Felipe, Baja California, Mexico': (31.0269529, -114.839607)}

In [2]:
import requests

def osrm_route(lat1, lon1, lat2, lon2):
    url = (
        "http://router.project-osrm.org/route/v1/driving/"
        f"{lon1},{lat1};{lon2},{lat2}"
        "?overview=false"
    )
    r = requests.get(url)
    r.raise_for_status()
    data = r.json()
    route = data["routes"][0]
    distance_km = route["distance"] / 1000      # meters → km
    duration_h = route["duration"] / 3600        # seconds → hours
    return distance_km, duration_h

In [3]:
import pandas as pd
import numpy as np

matrix = pd.DataFrame(index=cities, columns=cities, dtype=float)

for city_a in cities:
    lat1, lon1 = coords[city_a]
    for city_b in cities:
        if city_a == city_b:
            matrix.loc[city_a, city_b] = 0
        else:
            lat2, lon2 = coords[city_b]
            _, duration_h = osrm_route(lat1, lon1, lat2, lon2)
            matrix.loc[city_a, city_b] = duration_h

matrix

,"Tijuana, Baja California, Mexico","Mexicali, Baja California, Mexico","Ensenada, Baja California, Mexico","Tecate, Baja California, Mexico","Rosarito, Baja California, Mexico","San Quintin, Baja California, Mexico","San Felipe, Baja California, Mexico"
"Tijuana, Baja California, Mexico",0.000000,2.620611,1.620917,0.744472,0.485056,4.084833,5.113306
"Mexicali, Baja California, Mexico",2.400417,0.000000,3.375028,1.799056,2.639111,5.882806,2.971056
"Ensenada, Baja California, Mexico",1.522667,3.590694,0.000000,1.700944,1.179750,2.543250,3.653194
"Tecate, Baja California, Mexico",0.711139,2.000306,1.679000,0.000000,0.949833,4.186778,4.751583
"Rosarito, Baja California, Mexico",0.360139,2.839889,1.264722,0.941667,0.000000,3.728639,4.757111
"San Quintin, Baja California, Mexico",3.846250,5.954333,2.393056,4.064583,3.503333,0.000000,5.096278
"San Felipe, Baja California, Mexico",5.025972,2.948139,3.670333,4.649139,4.683056,5.257806,0.000000


In [4]:
import numpy as np

sym_matrix = pd.DataFrame(
    np.maximum(matrix.values, matrix.values.T),
    index=cities,
    columns=cities
)

sym_matrix

,"Tijuana, Baja California, Mexico","Mexicali, Baja California, Mexico","Ensenada, Baja California, Mexico","Tecate, Baja California, Mexico","Rosarito, Baja California, Mexico","San Quintin, Baja California, Mexico","San Felipe, Baja California, Mexico"
"Tijuana, Baja California, Mexico",0.000000,2.620611,1.620917,0.744472,0.485056,4.084833,5.113306
"Mexicali, Baja California, Mexico",2.620611,0.000000,3.590694,2.000306,2.839889,5.954333,2.971056
"Ensenada, Baja California, Mexico",1.620917,3.590694,0.000000,1.700944,1.264722,2.543250,3.670333
"Tecate, Baja California, Mexico",0.744472,2.000306,1.700944,0.000000,0.949833,4.186778,4.751583
"Rosarito, Baja California, Mexico",0.485056,2.839889,1.264722,0.949833,0.000000,3.728639,4.757111
"San Quintin, Baja California, Mexico",4.084833,5.954333,2.543250,4.186778,3.728639,0.000000,5.257806
"San Felipe, Baja California, Mexico",5.113306,2.971056,3.670333,4.751583,4.757111,5.257806,0.000000


In [6]:
# sym_matrix is in HOURS
matrix_minutes = (sym_matrix * 60).round().astype(int)
matrix_minutes

,"Tijuana, Baja California, Mexico","Mexicali, Baja California, Mexico","Ensenada, Baja California, Mexico","Tecate, Baja California, Mexico","Rosarito, Baja California, Mexico","San Quintin, Baja California, Mexico","San Felipe, Baja California, Mexico"
"Tijuana, Baja California, Mexico",0,157,97,45,29,245,307
"Mexicali, Baja California, Mexico",157,0,215,120,170,357,178
"Ensenada, Baja California, Mexico",97,215,0,102,76,153,220
"Tecate, Baja California, Mexico",45,120,102,0,57,251,285
"Rosarito, Baja California, Mexico",29,170,76,57,0,224,285
"San Quintin, Baja California, Mexico",245,357,153,251,224,0,315
"San Felipe, Baja California, Mexico",307,178,220,285,285,315,0


In [7]:
matrix_minutes.to_csv("bc_tsp_matrix.csv")